### List of features and test cases

1) What do you want to test?  
2) Integration or unit test?  
3) Structure of test case - 
    + Create inputs
    + Invoke the code to be tested
    + Compare the output
4) Handling expected failures

### Components to be tested

Get started with unit tests  
1) **Dope**  
2) Registry  
3) **Algorithms in arch/sklearn**  
4) get_best_model from optimizers/tune  
5) **classes from adapters**  
6) **layers and losses**  

### Test cases for Dope

 - Inputs -- primal_model, abstract_model, adapter  
 - Behaviour -- When provided with primal_model or abstract+adapter, 
   dope should provide a mapped dnn model.
 - mapped dnn model
 - Test expected failure cases as well
 
1) Test if all existing algorithms are supported by dope


### Test cases for Algorithms

#### Logistic regression

1) Compare `predict` values from primal and proxy  
    + Different variations and combinations
2) Generic config checks  
3) **Adding failure cases**  
4) Error handling

In [34]:
from mlsquare import dope

Using TensorFlow backend.
2019-07-26 12:02:58,032	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-26_12-02-58_7151/logs.
2019-07-26 12:02:58,146	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:28265 to respond...
2019-07-26 12:02:58,259	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:44235 to respond...
2019-07-26 12:02:58,261	INFO services.py:760 -- Starting Redis shard with 20.0 GB max memory.
2019-07-26 12:02:58,300	INFO services.py:1384 -- Starting the Plasma object store with 1.0 GB memory using /dev/shm.


In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

url = "../data/iris.csv"
data = pd.read_csv(url , delimiter=",", header=None, index_col=False)
class_name,index = np.unique(data.iloc[:,-1],return_inverse=True)
data.iloc[:,-1] = index
data = data.loc[data[4] != 2]
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.6, random_state=0)

In [36]:
from sklearn.linear_model import LogisticRegression
from mlsquare import dope

model = LogisticRegression()
m = dope(model)
trained_model = m.fit(x_train, y_train, epochs=500)

Transpiling your model to it's Deep Neural Network equivalent...
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
2019-07-26 12:02:59,453	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-07-26 12:02:59,454	INFO tune.py:211 -- Starting a new experiment.
2019-07-26 12:02:59,634	WARNING util.py:62 -- The `start_trial` operation took 0.17476582527160645 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.4/8.2 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.4/8.2 GB
Result logdir: /home/shakkeel/ray_results/experiment_name
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - train_model_0:	RUNNING

(pid=7443) Using TensorFlow backend.
(pid=7443) 2019-07-26 12:03:01,974	ERROR worker.py:1412 -- Calling ray.init() again after it has already been called.
(pid=7443) epochs ---  500
(pid=7443) WARNING:tensorflow:From /home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=7443) Instructions for updating:
(pid=7443) Colocations handled automatically by placer.
(pid=7443) WARNING:tensorflow:From /home/shakkeel/anaconda3/envs/test_imly/

2019-07-26 12:03:03,639	INFO ray_trial_executor.py:178 -- Destroying actor for trial train_model_0. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


Result for train_model_0:
  checkpoint: 'weights_tune_{''layer_1.units'': 1, ''layer_1.l1'': 0, ''layer_1.l2'':
    0, ''layer_1.activation'': ''sigmoid'', ''optimizer'': ''adam'', ''loss'': ''binary_crossentropy''}.h5'
  date: 2019-07-26_12-03-03
  done: false
  experiment_id: b93712234d8b4550a6775c2f454d6d46
  hostname: shakkeel-TUF-GAMING-FX504GD-FX80GD
  iterations_since_restore: 1
  mean_accuracy: 1.0
  node_ip: 192.168.1.4
  pid: 7443
  time_since_restore: 1.4653124809265137
  time_this_iter_s: 1.4653124809265137
  time_total_s: 1.4653124809265137
  timestamp: 1564122783
  timesteps_since_restore: 0
  training_iteration: 1
  
(pid=7443) 
(pid=7443) 32/40 [=======================>......] - ETA: 
(pid=7443) 40/40 [==============================] - 0s 409us/step
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.6/8.2 GB
Result logdir: /home/shakkeel/ray_results/experiment_name
Number of trials: 1 ({'TERMINATED': 1})
TE

In [15]:
m.save('test_onnx')

The maximum opset needed by this model is only 7.


In [2]:
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("test_onnx.onnx")  # load onnx model
# output = prepare(onnx_model).run(x_test)  # run the loaded model

/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/onnx_tf/common/__init__.py:87: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [6]:
import onnxruntime
sess = onnxruntime.InferenceSession('test_onnx.onnx')

In [7]:
input_name = sess.get_inputs()[0].name
print("Input name  :", input_name)
input_shape = sess.get_inputs()[0].shape
print("Input shape :", input_shape)
input_type = sess.get_inputs()[0].type
print("Input type  :", input_type)

Input name  : dense_2_input_0
Input shape : [1, 4]
Input type  : tensor(float)


In [8]:
output_name = sess.get_outputs()[0].name
print("Output name  :", output_name)  
output_shape = sess.get_outputs()[0].shape
print("Output shape :", output_shape)
output_type = sess.get_outputs()[0].type
print("Output type  :", output_type)

Output name  : dense_2_Sigmoid_01
Output shape : [1, 1]
Output type  : tensor(float)


In [24]:
_x = x_test.values
_x = _x.astype(np.float32)
_x

array([[5. , 3.4, 1.6, 0.4],
       [6.7, 3.1, 4.7, 1.5],
       [4.7, 3.2, 1.3, 0.2],
       [5.7, 2.8, 4.5, 1.3],
       [6.6, 3. , 4.4, 1.4],
       [5. , 2.3, 3.3, 1. ],
       [5.4, 3.9, 1.3, 0.4],
       [6.1, 2.8, 4.7, 1.2],
       [6.5, 2.8, 4.6, 1.5],
       [5.7, 3. , 4.2, 1.2],
       [5.5, 2.3, 4. , 1.3],
       [5.8, 2.6, 4. , 1.2],
       [6. , 2.9, 4.5, 1.5],
       [4.3, 3. , 1.1, 0.1],
       [5. , 3.4, 1.5, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [4.6, 3.6, 1. , 0.2],
       [4.8, 3.4, 1.9, 0.2],
       [5.5, 4.2, 1.4, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [6. , 2.2, 4. , 1. ],
       [4.6, 3.1, 1.5, 0.2],
       [6.1, 2.8, 4. , 1.3],
       [4.8, 3. , 1.4, 0.3],
       [5.3, 3.7, 1.5, 0.2],
       [4.6, 3.4, 1.4, 0.3],
       [5.7, 2.8, 4.1, 1.3],
       [5.8, 2.7, 3.9, 1.2],
       [6.8, 2.8, 4.8, 1.4],
       [5. , 2. , 3.5, 1. ],
       [5.5, 2.4, 3.8, 1.1],
       [5.5, 2.6, 4.4, 1.2],
       [6.2, 2.2, 4.5, 1.5],
       [6.4, 3

In [25]:
result = sess.run([output_name], {input_name: _x})

In [55]:
a = (result[0]>0.6).astype(np.int)

In [56]:
b = trained_model.predict_classes(x_test)

In [68]:
np.random.random((4,5)).shape

(4, 5)

In [62]:
from scipy import stats
stats.chisquare(a.reshape(60)+1,b.reshape(60)+1)

Power_divergenceResult(statistic=3.0, pvalue=1.0)

In [61]:
a.reshape(60)

array([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [28]:
from scipy import stats

model.fit(x_train, y_train)
pred1 = model.predict(x_test)

pred2 = m.final_model.predict_classes(x_test)
pred2 = pred2.reshape(60,)
pred2 = pred2.astype('int64')
pred2 += 1
pred1 += 1
# pred1 = pred1.astype('float64')
stats.chisquare(pred2,pred1)

/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Power_divergenceResult(statistic=1.0, pvalue=1.0)

In [22]:
pred1+1

array([1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2])

In [31]:
# stats.chisquare([16, 8, 16, 14, 12, 12], f_exp=[16, 16, 16, 16, 16, 8])
stats.chi2

In [32]:
stats.chisquare

<function scipy.stats.stats.chisquare(f_obs, f_exp=None, ddof=0, axis=0)>

2019-07-24 08:51:30,913	ERROR worker.py:1780 -- The node with client ID 3520b7657a79686fc9725a17b83b1a64c7cf3c14 has been marked dead because the monitor has missed too many heartbeats from it.


(pid=9381) Traceback (most recent call last):
(pid=9381)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=9381)     ray.worker.global_worker.main_loop()
(pid=9381)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1034, in main_loop
(pid=9381)     task = self._get_next_task_from_local_scheduler()
(pid=9381)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1017, in _get_next_task_from_local_scheduler
(pid=9381)     task = self.raylet_client.get_task()
(pid=9381)   File "python/ray/_raylet.pyx", line 244, in ray._raylet.RayletClient.get_task
(pid=9379) Traceback (most recent call last):
(pid=9379)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=9379)     ray.worker.global_worker.main_loop()
(pid=9379)   File "/home/shakkeel/

In [34]:

np.random.seed(12345678) 

rvs1 = stats.norm.rvs(loc=5,scale=10,size=500)
rvs2 = (stats.norm.rvs(loc=5,scale=10,size=500) +
        stats.norm.rvs(scale=0.2,size=500))
stats.ttest_rel(rvs1,rvs2)

rvs3 = (stats.norm.rvs(loc=8,scale=10,size=500) +
        stats.norm.rvs(scale=0.2,size=500))
stats.ttest_rel(rvs1,rvs3)

Ttest_relResult(statistic=-3.9995108708727924, pvalue=7.308240219166128e-05)

In [40]:
type(rvs1)

numpy.ndarray

In [42]:
rvs2.dtype

dtype('float64')

In [7]:
from mlsquare.imly import registry

model_skeleton, _ = registry[('sklearn', 'LogisticRegression')]['default']
default_params = model_skeleton.get_params()
test_params = {'new_param1': 1, 'new_param2':2}
model_skeleton.update_params(test_params)
# default_params.update(test_params)

In [11]:
default_params.update(test_params)
default_params

{'layer_1.units': 1,
 'layer_1.l1': 0,
 'layer_1.l2': 0,
 'layer_1.activation': 'sigmoid',
 'optimizer': 'adam',
 'loss': 'binary_crossentropy',
 'new_param1': 1,
 'new_param2': 2}

In [12]:
model_skeleton.get_params() == default_params

True

2019-07-20 09:45:37,552	ERROR worker.py:1780 -- The node with client ID 72d68030b26203cb162235b3e84dbbd45686f8e5 has been marked dead because the monitor has missed too many heartbeats from it.


(pid=28486) Traceback (most recent call last):
(pid=28486)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=28486)     ray.worker.global_worker.main_loop()
(pid=28486)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1034, in main_loop
(pid=28486)     task = self._get_next_task_from_local_scheduler()
(pid=28486)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1017, in _get_next_task_from_local_scheduler
(pid=28486)     task = self.raylet_client.get_task()
(pid=28486)   File "python/ray/_raylet.pyx", line 244, in ray._raylet.RayletClient.get_task
(pid=28489) Traceback (most recent call last):
(pid=28489)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=28489)     ray.worker.global_worker.main_loop()
(pid=28489)   File "/ho

Exception in thread ray_import_thread:
Traceback (most recent call last):
  File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/redis/connection.py", line 182, in _read_from_socket
    data = recv(self._sock, socket_read_size)
  File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/redis/_compat.py", line 58, in recv
    return sock.recv(*args, **kwargs)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/import_thread.py", line 70, in _run
    msg = import_pubsub_client.get_message()
  File "/home/

In [6]:
from scipy import stats

rvs1 = stats.norm.rvs(loc=5,scale=10,size=500)

rvs1.shape

(500,)

#### Linear Regression

In [47]:
from mlsquare import dope
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

model = LinearRegression()
diabetes = load_diabetes()

X = diabetes.data
sc = StandardScaler()
X = sc.fit_transform(X)
Y = diabetes.target
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

m = dope(model)

Transpiling your model to it's Deep Neural Network equivalent...


In [48]:
m.fit(x_train, y_train)

2019-07-22 17:49:46,466	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-07-22 17:49:46,467	INFO tune.py:211 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.2/8.2 GB



2019-07-22 17:49:49,715	WARNING util.py:62 -- The `start_trial` operation took 3.2396037578582764 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.2/8.2 GB
Result logdir: /home/shakkeel/ray_results/experiment_name
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - train_model_0:	RUNNING

(pid=27081) Using TensorFlow backend.
(pid=27081) 2019-07-22 17:52:46,724	ERROR worker.py:1412 -- Calling ray.init() again after it has already been called.
(pid=27081) epochs ---  250
(pid=27081) WARNING:tensorflow:From /home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=27081) Instructions for updating:
(pid=27081) Colocations handled automatically by placer.
(pid=27081) WARNING:tensorflow:From /home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is 

2019-07-22 17:54:01,414	INFO ray_trial_executor.py:178 -- Destroying actor for trial train_model_0. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


Result for train_model_0:
  checkpoint: 'weights_tune_{''layer_1.units'': 1, ''layer_1.l1'': 0, ''layer_1.l2'':
    0, ''layer_1.activation'': ''linear'', ''optimizer'': ''adam'', ''loss'': ''mse''}.h5'
  date: 2019-07-22_17-54-01
  done: false
  experiment_id: 66077dd556fb4b198cc914a98f3a597a
  hostname: shakkeel-TUF-GAMING-FX504GD-FX80GD
  iterations_since_restore: 1
  mean_accuracy: 0.0
  node_ip: 192.168.1.4
  pid: 27081
  time_since_restore: 20.23278260231018
  time_this_iter_s: 20.23278260231018
  time_total_s: 20.23278260231018
  timestamp: 1563798241
  timesteps_since_restore: 0
  training_iteration: 1
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.5/8.2 GB
Result logdir: /home/shakkeel/ray_results/experiment_name
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - train_model_0:	RUNNING, [4 CPUs, 0 GPUs], [pid=27081], 20 s, 1 iter, 0 acc

== Status ==
Using FIFO scheduling algorithm.
Resources requested

In [53]:
from scipy import stats

model.fit(x_train, y_train)
pred1 = model.predict(x_test)

pred2 = m.final_model.predict_classes(x_test)
pred2 = pred2.reshape(266,)
# pred2 = pred2.astype('float64')
# pred1 = pred1.astype('float64')
stats.ttest_rel(pred2,pred1, nan_policy='raise')

Ttest_relResult(statistic=-45.95922634383876, pvalue=2.920898371717077e-128)

In [52]:
from mlsquare import registry
import numpy as np
from keras.utils import to_categorical
from sklearn.tree import DecisionTreeClassifier
mock_model, _ = registry[('sklearn', 'DecisionTreeClassifier')]['default']
mock_model.y = np.array([[0,1], [0,1]])
mock_model.X = np.array([[0,1], [0,1]])
# y_train = to_categorical(y_train)
primal = DecisionTreeClassifier()
primal.fit(x_train, y_train)
mock_model.primal = primal
keras_model = mock_model.create_model()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
from mlsquare import dope
from sklearn.tree import DecisionTreeClassifier
dope(DecisionTreeClassifier(), version='random')
# # UserWarning('aa')
# import sys
# print('bkah ', file=sys.stderr)

TypeError: Model type `<class 'sklearn.tree.tree.DecisionTreeClassifier'>` is not supported by mlsquare yet.

In [3]:
from sklearn.linear_model import LogisticRegression
from mlsquare import registry, dope
model = LogisticRegression()
abstract_model, adapter = registry[('sklearn', 'LogisticRegression')]['default']
m = dope(model, abstract_model=abstract_model, adapt=adapter)

Transpiling your model to it's Deep Neural Network equivalent...


In [1]:

dope('ddd')

NameError: name 'dope' is not defined

### Notes on sklearn/numpy test cases

- t test
   + Can ttest be used 
- chi square

---

In [1]:
import mlsquare

Using TensorFlow backend.
2019-08-07 17:44:25,666	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-07_17-44-25_11428/logs.
2019-08-07 17:44:25,773	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:35625 to respond...
2019-08-07 17:44:25,893	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:63716 to respond...
2019-08-07 17:44:25,900	INFO services.py:760 -- Starting Redis shard with 20.0 GB max memory.
2019-08-07 17:44:25,918	INFO services.py:1384 -- Starting the Plasma object store with 1.0 GB memory using /dev/shm.


In [8]:
mlsquare.registry.data.keys()

dict_keys([('sklearn', 'LogisticRegression'), ('sklearn', 'LinearRegression'), ('sklearn', 'Ridge'), ('sklearn', 'Lasso'), ('sklearn', 'ElasticNet'), ('sklearn', 'LinearSVC'), ('sklearn', 'SVC'), ('sklearn', 'DecisionTreeClassifier')])

2019-08-08 09:11:04,101	ERROR worker.py:1780 -- The node with client ID 11693efe1e3e3b3fd96d4413a026d8017cbb80cf has been marked dead because the monitor has missed too many heartbeats from it.


(pid=11472) Traceback (most recent call last):
(pid=11472)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=11472)     ray.worker.global_worker.main_loop()
(pid=11472)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1034, in main_loop
(pid=11472)     task = self._get_next_task_from_local_scheduler()
(pid=11472)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1017, in _get_next_task_from_local_scheduler
(pid=11472)     task = self.raylet_client.get_task()
(pid=11472)   File "python/ray/_raylet.pyx", line 244, in ray._raylet.RayletClient.get_task
(pid=11470) Traceback (most recent call last):
(pid=11470)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=11470)     ray.worker.global_worker.main_loop()
(pid=11470)   File "/ho

In [2]:
from mlsquare.imly import dope
from sklearn.linear_model import LogisticRegression


In [4]:

model = LogisticRegression()
m = dope(model)

hasattr(m, 'score')

Transpiling your model to it's Deep Neural Network equivalent...


True

In [7]:
from mlsquare.imly import registry

In [8]:
_, adapter = registry[('sklearn', 'LogisticRegression')]['default']

In [10]:
adapter.__name__

'SklearnKerasClassifier'

In [80]:
class test():
    def __getitem__(self,x):
        pass
testtt= test()

In [81]:
from collections import Container
isinstance(testtt, dict)

False

In [24]:
from mlsquare import dope
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
m = dope(model)
# m.save('testing_pickle')

Transpiling your model to it's Deep Neural Network equivalent...


In [4]:
import pickle

with open(r"test_onnx.pkl", "rb") as input_file:
        e = pickle.load(input_file)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
help(e)

Help on Sequential in module keras.engine.sequential object:

class Sequential(keras.engine.training.Model)
 |  Linear stack of layers.
 |  
 |  # Arguments
 |      layers: list of layers to add to the model.
 |  
 |  # Example
 |  
 |  ```python
 |  # Optionally, the first layer can receive an `input_shape` argument:
 |  model = Sequential()
 |  model.add(Dense(32, input_shape=(500,)))
 |  
 |  # Afterwards, we do automatic shape inference:
 |  model.add(Dense(32))
 |  
 |  # This is identical to the following:
 |  model = Sequential()
 |  model.add(Dense(32, input_dim=500))
 |  
 |  # And to the following:
 |  model = Sequential()
 |  model.add(Dense(32, batch_input_shape=(None, 500)))
 |  
 |  # Note that you can also omit the `input_shape` argument:
 |  # In that case the model gets built the first time you call `fit` (or other
 |  # training and evaluation methods).
 |  model = Sequential()
 |  model.add(Dense(32))
 |  model.add(Dense(32))
 |  model.compile(optimizer=optimizer, lo

In [ ]:
file = open("testing_pickle.obj",'r')
object_file = pickle.load(file)

In [2]:
import pickle

data1 = {'a': [1, 2.0, 3, 4+6j],
         'b': ('string', u'Unicode string'),
         'c': None}

selfref_list = [1, 2, 3]
selfref_list.append(selfref_list)

In [11]:
selfref_list[3][3]

[1, 2, 3, [...]]

In [12]:
output = open('data.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(data1, output)

# Pickle the list using the highest protocol available.
pickle.dump(selfref_list, output, -1)

output.close()

In [16]:
import pprint, pickle

pkl_file = open('data.pkl', 'rb')

data1 = pickle.load(pkl_file)
pprint.pprint(data1)

data2 = pickle.load(pkl_file)
pprint.pprint(data2)

pkl_file.close()

{'a': [1, 2.0, 3, (4+6j)], 'b': ('string', 'Unicode string'), 'c': None}


In [1]:
from mlsquare import registry

Using TensorFlow backend.
2019-08-12 18:22:36,398	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-12_18-22-36_659/logs.
2019-08-12 18:22:36,518	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:14380 to respond...
2019-08-12 18:22:36,640	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:11421 to respond...
2019-08-12 18:22:36,642	INFO services.py:760 -- Starting Redis shard with 20.0 GB max memory.
2019-08-12 18:22:36,676	INFO services.py:1384 -- Starting the Plasma object store with 1.0 GB memory using /dev/shm.


In [2]:
proxy_model,_ = registry[('sklearn', 'LogisticRegression')]['default']

In [3]:
proxy_model

In [3]:
import pickle

file_details = open('proxy_data.pkl', 'wb')
 
pickle.dump(proxy_model, file_details)

PicklingError: Can't pickle <class 'mlsquare.architectures.sklearn.LogisticRegression'>: it's not the same object as mlsquare.architectures.sklearn.LogisticRegression

In [4]:
pickle

<module 'pickle' from '/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/pickle.py'>

2019-08-13 11:33:17,542	ERROR worker.py:1780 -- The node with client ID cb0653dd5111d5c5f2793a66cb612bbb2a266b5d has been marked dead because the monitor has missed too many heartbeats from it.


(pid=709) Traceback (most recent call last):
(pid=709)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=709)     ray.worker.global_worker.main_loop()
(pid=709)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1034, in main_loop
(pid=709)     task = self._get_next_task_from_local_scheduler()
(pid=709)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1017, in _get_next_task_from_local_scheduler
(pid=709)     task = self.raylet_client.get_task()
(pid=709)   File "python/ray/_raylet.pyx", line 244, in ray._raylet.RayletClient.get_task
(pid=708) Traceback (most recent call last):
(pid=708)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=708)     ray.worker.global_worker.main_loop()
(pid=708)   File "/home/shakkeel/anaconda3/en

In [4]:
import dill

file_details = open('proxy_data.pkl', 'wb')
 
pickle.dump(proxy_model, file_details)

PicklingError: Can't pickle <class 'mlsquare.architectures.sklearn.LogisticRegression'>: it's not the same object as mlsquare.architectures.sklearn.LogisticRegression

2019-08-12 16:38:49,922	ERROR worker.py:1780 -- The node with client ID 29f30a409e46e024b27fc22f7993d98d38d2c78f has been marked dead because the monitor has missed too many heartbeats from it.


(pid=29483) Traceback (most recent call last):
(pid=29483)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/workers/default_worker.py", line 111, in <module>
(pid=29483)     ray.worker.global_worker.main_loop()
(pid=29483)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1034, in main_loop
(pid=29483)     task = self._get_next_task_from_local_scheduler()
(pid=29483)   File "/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ray/worker.py", line 1017, in _get_next_task_from_local_scheduler
(pid=29483)     task = self.raylet_client.get_task()
(pid=29483)   File "python/ray/_raylet.pyx", line 244, in ray._raylet.RayletClient.get_task
(pid=29483)   File "python/ray/_raylet.pyx", line 59, in ray._raylet.check_status
(pid=29483) Exception: [RayletClient] Raylet connection closed.
(pid=29483) 
(pid=29483) During handling of the above exception, another exception occurred:
(pid=29483) 
(pid=29483) T

In [16]:
class LogisticRegression():
    def __init__(self):
        # self.adapter = SklearnKerasClassifier
        self.module_name = 'sklearn'  # Rename the variable
        self.name = 'LogisticRegression'
        self.version = 'default'
        model_params = {'layer_1': {'units': 1, ## Make key name private - '_layer'
                                    'l1': 0,
                                    'l2': 0,
                                    'activation': 'sigmoid'},
                        'optimizer': 'adam',
                        'loss': 'binary_crossentropy'
                        }
        self.set_params(params=model_params, set_by='model_init')
        
    def set_params(self, **kwargs):
        pass

In [17]:
lr_instance = LogisticRegression()

In [11]:
type(lr_instance)

__main__.LogisticRegression

In [13]:
type(proxy_model)

mlsquare.architectures.sklearn.LogisticRegression

In [24]:
import pickle
file_details = open('proxy_data.pkl', 'wb')
 
pickle.dump(proxy_model, file_details)
file_details.close()

PicklingError: Can't pickle <class 'mlsquare.architectures.sklearn.LogisticRegression'>: it's not the same object as mlsquare.architectures.sklearn.LogisticRegression

In [21]:
file_details = open('proxy_data.pkl', 'rb')

loaded_data = pickle.load(file_details)

file_details.close()

In [23]:
help(loaded_data)

Help on LogisticRegression in module __main__ object:

class LogisticRegression(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  set_params(self, **kwargs)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __slotnames__ = []



In [30]:
a = getattr(proxy_model, '__qualname__', None)

In [36]:
name = proxy_model.__name__

AttributeError: 'LogisticRegression' object has no attribute '__name__'